In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install tensorflow==2.13.0

In [3]:
import tensorflow as tf
print("TensorFlow version:", tf.__version__)
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow import keras 
from keras import optimizers
from keras import applications
from keras.models import Model
from keras.models import load_model

TensorFlow version: 2.12.0


In [10]:
mnist = tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()


11490434/11490434 [==============================] - 1s 0us/step


In [29]:
type(x_train[0][0][0])

numpy.uint8

In [15]:
# Sequential is useful for stacking layers where each layer has one input tensor 
# and one output tensor. Layers are functions with a known mathematical structure 
# that can be reused and have trainable variables. Most TensorFlow models are
# composed of layers. This model uses the Flatten, Dense, and Dropout layers.

model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28,28)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(10)
])

In [32]:
# For each example, the model returns a vector of logits or log-odds scores, 
# one for each class.

predictions = model(x_train[:1]).numpy()

# The tf.nn.softmax function converts these logits to probabilities for each class: 
tf.nn.softmax(predictions).numpy()


array([[0., 0., 0., 0., 0., 0., 0., 0., 1., 0.]], dtype=float32)

In [34]:
# Define a loss function for training using losses.SparseCategoricalCrossentropy:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

# The loss function takes a vector of ground truth values and a vector of logits
# and returns a scalar loss for each example. This loss is equal to the negative
# log probability of the true class: The loss is zero if the model is sure of the
# correct class.

In [35]:
# This untrained model gives probabilities close to random (1/10 for each class),
# so the initial loss should be close to -tf.math.log(1/10) ~= 2.3.
loss_fn(y_train[:1], predictions).numpy()

438.18146

In [37]:
# Before you start training, configure and compile the model using Keras Model.compile.
# Set the optimizer class to adam, set the loss to the loss_fn function you defined
# earlier, and specify a metric to be evaluated for the model by setting the metrics
# parameter to accuracy.
model.compile(optimizer='adam',
              loss=loss_fn,
              metrics=['accuracy'])

### Train and evaluate the model

In [38]:
# Use the Model.fit method to adjust your model parameters and minimize the loss: 
model.fit(x_train, y_train, epochs=5)

Epoch 1/5
Cause: Unable to locate the source code of <function Model.make_train_function.<locals>.train_function at 0x7902834630a0>. Note that functions defined in certain environments, like the interactive Python shell, do not expose their source code. If that is the case, you should define them in a .py source file. If you are certain the code is graph-compatible, wrap the call using @tf.autograph.experimental.do_not_convert. Original error: could not get source code
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
1875/1875 [==============================] - 5s 2ms/step - loss: 2.4432 - accuracy: 0.7389
Epoch 2/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.5658 - accuracy: 0.8529
Epoch 3/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.4539 - accuracy: 0.8803
Epoch 4/5
1875/1875 [==============================] - 5s 2ms/step - loss: 0.3948 - accuracy: 0.8958
Epoch 5/5
1875/1875 [=================

In [39]:
# The Model.evaluate method checks the model's performance, usually on a validation set or test set.
model.evaluate(x_test,  y_test, verbose=2)

Cause: Unable to locate the source code of <function Model.make_test_function.<locals>.test_function at 0x790272ef2050>. Note that functions defined in certain environments, like the interactive Python shell, do not expose their source code. If that is the case, you should define them in a .py source file. If you are certain the code is graph-compatible, wrap the call using @tf.autograph.experimental.do_not_convert. Original error: could not get source code
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
313/313 - 0s - loss: 0.2874 - accuracy: 0.9397 - 462ms/epoch - 1ms/step


[0.2873843312263489, 0.9397000074386597]

In [40]:
# If you want your model to return a probability, you can wrap the trained model,
# and attach the softmax to it:
probability_model = tf.keras.Sequential([
  model,
  tf.keras.layers.Softmax()
])


probability_model(x_test[:5])

<tf.Tensor: shape=(5, 10), dtype=float32, numpy=
array([[0.0000000e+00, 4.8945771e-21, 2.0332876e-11, 4.5293188e-13,
        4.9698682e-23, 7.1345559e-16, 0.0000000e+00, 1.0000000e+00,
        3.5472360e-26, 2.1768643e-15],
       [1.5356455e-16, 1.3054944e-09, 1.0000000e+00, 2.3280184e-13,
        0.0000000e+00, 1.2906708e-21, 4.6287240e-20, 1.0765966e-19,
        8.9410205e-25, 0.0000000e+00],
       [1.8049276e-27, 1.0000000e+00, 8.5618057e-17, 4.4635979e-25,
        3.2449359e-26, 1.0045996e-15, 3.5536586e-17, 7.6811832e-13,
        6.6989707e-09, 5.3169085e-35],
       [9.9997818e-01, 1.4278572e-24, 2.0502755e-05, 1.9252584e-09,
        1.1850655e-13, 1.1764952e-07, 2.4682012e-07, 9.4471545e-07,
        2.1339719e-08, 5.5052602e-09],
       [4.1882301e-23, 1.4297512e-12, 5.6475596e-10, 8.7771228e-11,
        9.9927241e-01, 9.8131963e-12, 8.2167450e-17, 4.3304282e-04,
        3.8746903e-16, 2.9464788e-04]], dtype=float32)>